In [1]:
# Importing required modules.
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import plotly.plotly as py
from plotly.graph_objs import *

In [2]:
def create_features() :
    data = pd.read_csv("train.csv")
    data = data.dropna()
    # Transforming weekday into numbers.
    data = data.replace("Monday", 1)
    data = data.replace("Tuesday", 2)
    data = data.replace("Wednesday", 3)
    data = data.replace("Thursday", 4)
    data = data.replace("Friday", 5)
    data = data.replace("Saturday", 6)
    data = data.replace("Sunday", 7)
    descriptionDummies = pd.get_dummies(data.DepartmentDescription)
    descriptionDummies = descriptionDummies.apply(lambda x: x*data["ScanCount"])
    data.loc[data.ScanCount < 0, 'Return'] = 1
    data.loc[data.Return != 1, 'Return'] = 0
    data = data[["TripType", "VisitNumber", "Weekday", "ScanCount", "Return"]]
    dataPart1 = data.groupby("VisitNumber").agg({'Weekday': np.max, "TripType": np.max, 'ScanCount': np.sum, 'Return': np.max})
    dataPart2 = pd.concat([data.VisitNumber,descriptionDummies],axis=1)
    dataPart2 = dataPart2.groupby("VisitNumber").agg('sum')
    data = pd.concat([dataPart1, dataPart2], axis=1)
    return data

In [3]:
data = create_features()
data

,TripType,Return,Weekday,ScanCount,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
VisitNumber,,,,,,,,,,,,,,,,,,,,,
5,999,1.0,5,-1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,30,0.0,5,2,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,26,1.0,5,27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,8,0.0,5,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,8,0.0,5,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,35,0.0,5,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,41,0.0,5,7,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
15,21,0.0,5,9,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,6,0.0,5,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn import metrics
from sklearn import svm
from sklearn.cross_validation import train_test_split

C:\Users\Arun\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [6]:
Y = data.iloc[:,0]
X = data.iloc[:,1:]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
model = svm.SVC()
model.fit(X_train, Y_train)
# predict class labels for the test set
predicted = model.predict(X_test)
# generate evaluation metrics
print metrics.accuracy_score(Y_test, predicted)
print metrics.confusion_matrix(Y_test, predicted)
print metrics.classification_report(Y_test, predicted)

0.680990274094
[[ 991    0    0 ...,    0    0    1]
 [   0    0   95 ...,    0    0    0]
 [   1    0  801 ...,    0    1    0]
 ..., 
 [   0    0   11 ...,    5    5    0]
 [   0    0    3 ...,    5   79    0]
 [ 232    0   33 ...,    1    0 1837]]
             precision    recall  f1-score   support

          3       0.77      0.95      0.85      1048
          4       0.00      0.00      0.00       103
          5       0.59      0.81      0.68       992
          6       0.68      0.48      0.56       408
          7       0.69      0.66      0.67      1688
          8       0.79      0.84      0.81      3752
          9       0.66      0.86      0.75      2810
         12       0.00      0.00      0.00        77
         15       0.60      0.39      0.48       298
         18       0.42      0.47      0.44       156
         19       0.24      0.03      0.06       116
         20       0.58      0.60      0.59       179
         21       0.61      0.71      0.66       206
      

C:\Users\Arun\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

